In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Tuple, List, Dict
import random
import re
from textblob import TextBlob
import time
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Device and model configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "./fine_tuned_personality_bot/"  # Update with your model path

class PersonalityBot:
    def __init__(self, model_path: str = MODEL_PATH):
        self.model_path = model_path
        self.model, self.tokenizer = self.setup_model()
        
    def setup_model(self) -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
        """Initialize and configure the model and tokenizer."""
        logger.info(f"Setting up model from {self.model_path}")
    
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found at {self.model_path}")
    
        try:
            tokenizer = AutoTokenizer.from_pretrained(self.model_path, use_fast=True)
            tokenizer.pad_token = tokenizer.eos_token
    
        except Exception as e:
            logger.error(f"Tokenizer loading failed: {str(e)}")
            raise
    
        try:
            model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                low_cpu_mem_usage=True,
                device_map="auto"
            )
            model.eval()
            logger.info("Model setup completed successfully")
            return model, tokenizer
    
        except Exception as e:
            logger.error(f"Model loading failed: {str(e)}")
            raise

    def categorize_prompt(self, prompt: str) -> str:
        """Categorize input prompt for contextual response generation."""
        categories: Dict[str, List[str]] = {
            "market_analysis": [
                "price", "market", "chart", "analysis", "trend", "prediction",
                "bull", "bear", "trading", "volume"
            ],
            "tech_discussion": [
                "blockchain", "protocol", "layer", "scaling", "code", "development",
                "smart contract", "gas", "network"
            ],
            "defi": [
                "defi", "yield", "farming", "liquidity", "stake", "lending",
                "borrow", "apy", "tvl"
            ],
            "nft": [
                "nft", "art", "collectible", "mint", "opensea", "rarity",
                "floor price", "pfp"
            ],
            "culture": [
                "community", "dao", "governance", "vote", "proposal",
                "alpha", "degen", "fud", "fomo"
            ]
        }
        
        prompt_lower = prompt.lower()
        for category, keywords in categories.items():
            if any(keyword in prompt_lower for keyword in keywords):
                return category
        return "general"

    def generate_hook(self, category: str) -> str:
        """Generate category-specific attention hooks with an expanded list."""
        hooks = {
            "market_analysis": [
                "Market alert:", "Chart check:", "Price watch:",
                "Trading insight:", "Market alpha:",
                "Trend spotting:", "Candlelight stories:", "RSI deep dive:",
                "Volatility watch:", "Support level breakdown:",
                "Resistance zone spotted:", "Market movers:",
                "All eyes on the charts:", "Is this a bull trap?",
                "Breakout or fakeout?", "Today's key levels:"
            ],
            "tech_discussion": [
                "Tech deep dive:", "Builder's corner:", "Protocol watch:",
                "Dev update:", "Architecture take:",
                "Blockchain in focus:", "Gas fee breakdown:", "Scaling challenges:",
                "Layer 2 spotlight:", "New upgrade analysis:",
                "Consensus mechanism debate:", "Crypto tech wars:",
                "Network optimization insights:", "Codebase comparison:",
                "Innovator's edge:", "Protocol vulnerabilities exposed:"
            ],
            "defi": [
                "DeFi alpha:", "Yield watch:", "Smart money move:",
                "Protocol alert:", "TVL update:",
                "Farming frenzy:", "Liquidity trends:", "Borrowing breakdown:",
                "Stakeholder spotlight:", "APR vs APY debate:",
                "Risk-adjusted returns:", "What's your yield strategy?",
                "Stablecoin flow insights:", "Vault innovations:",
                "Lending protocol comparison:", "DeFi's next big move:"
            ],
            "nft": [
                "NFT alpha:", "Collection watch:", "Mint alert:",
                "Floor check:", "Digital art take:",
                "Art reveal:", "Rare trait spotted:", "Is this the next blue chip?",
                "Profile picture wars:", "Who's flipping this?",
                "NFT drama explained:", "Rarity analysis:",
                "Auction insights:", "Utility vs hype debate:",
                "Next-gen collectibles:", "Art meets utility:"
            ],
            "culture": [
                "Culture take:", "DAO watch:", "Governance alert:",
                "Community vibe:", "Alpha leak:",
                "The crypto ethos:", "FOMO or FUD?", "Web3 lifestyle:",
                "Building the future:", "Influencer drama explained:",
                "Community-driven innovation:", "DAO proposal debates:",
                "Web3's cultural revolution:", "Crypto memes decoded:",
                "The rise of governance tokens:", "Who else is building?"
            ],
            "general": [
                "Hot take:", "Unpopular opinion:", "Plot twist:",
                "Real talk:", "Quick thought:",
                "Imagine this:", "What if I told you:", "Could this be true?",
                "Something to chew on:", "Here’s an idea:",
                "Change my mind:", "Big picture time:",
                "Food for thought:", "The future is calling:", "What comes next?",
                "Let’s break it down:"
            ]
        }
        
        category_hooks = hooks.get(category, hooks["general"])
        return random.choice(category_hooks) if random.random() < 0.2 else ""
    
    def add_emojis(self, text: str, category: str) -> str:
        """Add contextual emojis based on content category, with limited frequency."""
        emoji_sets = {
            "market_analysis": ["📈", "📊", "💹", "📉", "💸", "🎯", "📱"],
            "tech_discussion": ["⚡️", "🔧", "💻", "🛠️", "🔨", "🧮", "🔋"],
            "defi": ["🏦", "💰", "🏧", "💳", "🔄", "⚖️", "🎰"],
            "nft": ["🎨", "🖼️", "🎭", "🎪", "🎟️", "🎮", "🃏"],
            "culture": ["🌐", "🤝", "🗣️", "🎭", "🎪", "🎯", "🎲"],
            "general": ["🚀", "💎", "🌙", "🔥", "💡", "🎯", "⭐️"]
        }
        
        # Add emojis with 20% probability
        if random.random() > 0.2:
            return text
    
        category_emojis = emoji_sets.get(category, emoji_sets["general"])
        emoji_count = random.randint(1, 2)
        chosen_emojis = random.sample(category_emojis, emoji_count)
        
        return f"{text} {' '.join(chosen_emojis)}"

    def generate_engagement_phrase(self, category: str) -> str:
        """Generate contextual engagement prompts."""
        phrases = {
            "market_analysis": [
                "What's your price target?",
                "Bulls or bears?",
                "Who's buying this dip?",
                "Thoughts on this setup?"
            ],
            "tech_discussion": [
                "Devs, thoughts?",
                "Valid architecture?",
                "Spotted any issues?",
                "Who's building similar?"
            ],
            "defi": [
                "What's your yield strategy?",
                "Aping in?",
                "Found better rates?",
                "Risk level?"
            ],
            "nft": [
                "Cope or hope?",
                "Floor predictions?",
                "Minting this?",
                "Art or utility?"
            ],
            "culture": [
                "Based or nah?",
                "Who else sees this?",
                "Your governance take?",
                "DAO voters wya?"
            ],
            "general": [
                "Thoughts?",
                "Based?",
                "Who's with me?",
                "Change my mind."
            ]
        }
        
        category_phrases = phrases.get(category, phrases["general"])
        return random.choice(category_phrases) if random.random() < 0.3 else ""

    def add_hashtags(self, text: str, category: str) -> str:
        """Add relevant hashtags based on content and character limit, with limited frequency."""
        hashtags = {
            "market_analysis": [
                "#CryptoTrading", "#TechnicalAnalysis", "#CryptoMarkets",
                "#Trading", "#Charts", "#PriceAction"
            ],
            "tech_discussion": [
                "#Blockchain", "#CryptoTech", "#Web3Dev", "#DLT",
                "#SmartContracts", "#BuilderSpace"
            ],
            "defi": [
                "#DeFi", "#YieldFarming", "#Staking", "#DeFiSeason",
                "#PassiveIncome", "#DeFiYield"
            ],
            "nft": [
                "#NFTs", "#NFTCommunity", "#NFTCollector", "#NFTArt",
                "#NFTProject", "#TokenizedArt"
            ],
            "culture": [
                "#CryptoCulture", "#DAOs", "#Web3", "#CryptoTwitter",
                "#CryptoLife", "#BuildingWeb3"
            ],
            "general": [
                "#Crypto", "#Web3", "#Bitcoin", "#Ethereum",
                "#CryptoTwitter", "#BuildingTheFuture"
            ]
        }
    
        # Add hashtags with 40% probability
        if random.random() > 0.2:
            return text
    
        remaining_chars = 280 - len(text)
        if remaining_chars < 15:  # Not enough space for hashtags
            return text
    
        category_hashtags = hashtags.get(category, hashtags["general"])
        selected_hashtags = []
        
        # Add 1-2 hashtags while respecting character limit
        for _ in range(random.randint(1, 2)):
            if not category_hashtags or remaining_chars < 15:
                break
            hashtag = random.choice(category_hashtags)
            if len(hashtag) + 1 <= remaining_chars:
                selected_hashtags.append(hashtag)
                category_hashtags.remove(hashtag)
                remaining_chars -= len(hashtag) + 1
    
        return f"{text} {' '.join(selected_hashtags)}"
    
    def clean_response(self, text: str, category: str) -> str:
        """Clean and format the response for Twitter."""
        # Remove URLs and excessive whitespace
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
    
        # Remove leading and trailing quotation marks
        text = text.strip('"\'“”')
    
        # Replace multiple internal quotes with single quotes
        text = re.sub(r'"+', '"', text)
        text = re.sub(r"'+", "'", text)
    
        # Correct unbalanced quotation marks
        def balance_quotes(s):
            quote_chars = ['"', "'"]
            for quote in quote_chars:
                if s.count(quote) % 2 != 0:
                    s = s.replace(quote, '')  # Remove unmatched quotes
            return s
    
        text = balance_quotes(text)
    
        # Ensure the text ends with proper punctuation
        if text and text[-1] not in '.!?':
            text += '.'
    
        return text



    def get_fallback(self, category: str) -> str:
        """Generate category-specific fallback responses."""
        fallbacks = {
            "market_analysis": [
                "Charts looking juicy today! Anyone else seeing this setup? 📈",
                "Market's giving mixed signals but the volume tells a different story 👀"
            ],
            "tech_discussion": [
                "Sometimes the best protocols are the ones no one's talking about yet 🛠️",
                "Imagine still building without considering Layer 2 scaling 💻"
            ],
            "defi": [
                "Your yields are only as good as your risk management 🏦",
                "DeFi summer never ended, we just got better at farming 🌾"
            ],
            "nft": [
                "Art is subjective, but floor prices aren't 🎨",
                "Your NFT portfolio tells a story. Make it a good one 🖼️"
            ],
            "culture": [
                "Web3 culture is what we make it. Build accordingly 🌐",
                "Sometimes the real alpha is the friends we made along the way 🤝"
            ],
            "general": [
                "Just caught myself thinking about the future of crypto while making coffee ☕️",
                "Your portfolio is only as strong as your conviction 💎"
            ]
        }
        
        category_fallbacks = fallbacks.get(category, fallbacks["general"])
        return random.choice(category_fallbacks)

    def filter_tone(self, response: str) -> str:
        """Filter response tone and adjust if needed."""
        sentiment = TextBlob(response).sentiment
        
        if sentiment.polarity < -0.3:
            return self.get_fallback("general")
        
        if sentiment.subjectivity > 0.8:
            # Too subjective, add a disclaimer
            return f"Not financial advice but... {response}"
                
        return response

    def generate_response(self, prompt: str) -> str:
        """Generate a complete Twitter-ready response."""
        category = self.categorize_prompt(prompt)
    
        instruction = (
            "You are a crypto and finance expert with a sharp sense of humor, blending the witty sarcasm of George Hotz with the storytelling flair of Theo Von. "
            "Your goal is to craft engaging, funny, and insightful tweets that educate your audience using appropriate slang and jargon. "
            "Each tweet should be coherent, make logical sense, and provide a clear takeaway or punchline. "
            "Avoid overusing slang—use it where it feels natural. "
            "Respond to the following prompt:\n\n"
        )
    
        examples = (
            "Prompt: What's your take on Bitcoin as digital gold?\n"
            "Tweet: Bitcoin as digital gold? Nah, it's more like digital real estate in the metaverse—except everyone's still arguing over the property lines. Who's still buying up the neighborhood? 🚀 #Bitcoin #Crypto\n\n"
            "Prompt: Explain staking in the context of DeFi but make it funny.\n"
            "Tweet: Staking in DeFi is like putting your money on a treadmill—you lock it up, it works out, and somehow you end up with more than just sweaty tokens. Gains on gains! 🏋️‍♂️ #DeFi #Staking\n\n"
        )
    
        context = f"{instruction}{examples}Prompt: {prompt}\nTweet:"
    
        inputs = self.tokenizer(
            context,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024  # Increased to accommodate longer context
        ).to(device)
    
        try:
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=80,
                    do_sample=True,
                    temperature=0.7,
                    top_k=50,
                    top_p=0.9,
                    repetition_penalty=1.5,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
    
            generated_text = self.tokenizer.decode(
                outputs[0],
                skip_special_tokens=True
            )
    
            # Extract the tweet from the generated text
            response = generated_text.split("Tweet:")[-1].strip().split("\n")[0]
    
            if not response or len(response) < 20:
                return self.get_fallback(category)
    
            # Apply enhancements
            response = self.clean_response(response, category)
            response = self.filter_tone(response)
            response = self.add_emojis(response, category)
            response = self.add_hashtags(response, category)
    
            logger.info(f"Generated response: {response}")
    
            return response[:280]  # Ensure the response fits within Twitter's character limit
    
        except Exception as e:
            logger.error(f"Error generating response: {str(e)}")
            return self.get_fallback(category)
    


def main():
    """Main execution function."""
    try:
        bot = PersonalityBot()
        logger.info("Bot initialized successfully")
        
        # Test prompts
        test_prompts = [
            "How do gas fees compare between Ethereum and Solana?",
            "Tell me why NFTs are either a scam or the future.",
            "What's your take on Bitcoin as digital gold?",
            "Explain staking in the context of DeFi but make it funny.",
            "Give me a wild prediction for crypto in 2030.",
            "What's a DAO, and should I care?",
            "Can you explain why everyone's obsessed with Layer 2 solutions?",
            "What's the wildest crypto meme you've seen this week?",
            "How do I avoid FOMO during bull runs?",
            "Is it too late to get into crypto? Be honest!",
            "How would you describe Web3 to someone who barely uses the internet?",
            "What's your hottest take on decentralized finance?"
        ]
        
        # Test run with basic prompts
        logger.info("Starting test run with sample prompts...")
        print("\n=== Test Run with Sample Prompts ===")
        for i, prompt in enumerate(test_prompts, 1):
            print(f"\nPrompt {i}: {prompt}")
            start_time = time.time()
            response = bot.generate_response(prompt)
            elapsed_time = time.time() - start_time
            print("")
            print(f"Response: {response}")
            print(f"Runtime: {elapsed_time:.2f} seconds")
        
        # Interactive mode
        logger.info("Entering interactive mode...")
        print("\n=== Interactive Mode ===")
        print("Enter your prompts (type 'quit' to exit):")
        
        while True:
            user_prompt = input("\nYour prompt: ").strip()
            if user_prompt.lower() == 'quit':
                print("Exiting... Thanks for using CryptoPersonalityBot!")
                break
            if not user_prompt:
                print("Please enter a valid prompt!")
                continue
            try:
                start_time = time.time()
                response = bot.generate_response(user_prompt)
                elapsed_time = time.time() - start_time
                print(f"Response: {response}")
                print(f"Runtime: {elapsed_time:.2f} seconds")
            except Exception as e:
                logger.error(f"Error processing prompt: {str(e)}")
                print("Oops! Something went wrong. Please try again.")
    except Exception as e:
        logger.error(f"Application error: {str(e)}")
        print("Critical error occurred. Please check the logs for details.")

if __name__ == "__main__":
    main()


2024-11-19 14:16:22,879 - INFO - Setting up model from ./fine_tuned_personality_bot/
2024-11-19 14:16:24,159 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2024-11-19 14:16:40,471 - WARNING - Some parameters are on the meta device because they were offloaded to the cpu.
2024-11-19 14:16:40,473 - INFO - Model setup completed successfully
2024-11-19 14:16:40,473 - INFO - Bot initialized successfully
2024-11-19 14:16:40,473 - INFO - Starting test run with sample prompts...



=== Test Run with Sample Prompts ===

Prompt 1: How do gas fees compare between Ethereum and Solana?


2024-11-19 14:17:05,610 - INFO - Generated response: Gas fee wars - who needs friends when we have high transaction costs?! Just kidding... sorta 😜 But seriously, if u wanna transact fast & cheaply w/ @Solana (now Sol), expect ~$0.00005 per tx compared 2 Eth (~ $3-$4). Mind blown yet 💸🔥.


...


Your prompt:  Would ETH and SOL go on a date?


2024-11-19 14:21:58,908 - INFO - Generated response: So I asked Ethereum (ETH) if she wanted to grab coffee w/ Solana (SOL). She said sure...but only after he paid for her gas fees & promised not 2 "fork" things mid-date 😳💸 Can we say they had an interesting conversation about scalability vs sustainability 💬👫? #BuildingTheFuture


Response: So I asked Ethereum (ETH) if she wanted to grab coffee w/ Solana (SOL). She said sure...but only after he paid for her gas fees & promised not 2 "fork" things mid-date 😳💸 Can we say they had an interesting conversation about scalability vs sustainability 💬👫? #BuildingTheFuture
Runtime: 24.75 seconds



Your prompt:  quit


Exiting... Thanks for using CryptoPersonalityBot!
